# Config

### Imports

In [503]:
import itertools
import os
import random
import shutil
import time
from tqdm.notebook import tqdm
import uuid
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
import matplotlib.pyplot as plt

In [504]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

### Constants

In [554]:
TRAIN_ON_TF = 'TF_31'
TRAIN_PROBE = 'A'
TEST_PROBE = 'B'
BATCH_SIZE = 64

In [555]:
DNA_BASES = 'ACGT'
RNA_BASES = 'ACGU'

In [556]:
PBM_DATA = "./data/dream5/pbm"

# Data preparation

### Read DREAM5 sequence data with binding scores

In [557]:
df_seq = pd.read_csv(f"{PBM_DATA}/sequences.tsv", sep='\t')
df_seq.head()

,Fold ID,Event ID,seq
0,B,MEreverse14075,TAAAACTATGAGGAAGGATTCAGGGTCGGACAGTGCCTGT
1,B,MEforward19438,CTTATGATCAGAAGCGGCTAGGTGTATTACATGTCCCTGT
2,B,MEforward19439,CCGCCGTAGGCCCCGAAACAGTACCAGACATGTAACCTGT
3,B,MEforward19436,GACCAAACGAGTCCTAGGATTCCAAGCGTTACGACCCTGT
4,B,MEforward19437,CGTTACGACGGAGTTTGGATCCCGAACTTATGATCCCTGT


In [558]:
df_targets = pd.read_csv(f"{PBM_DATA}/targets.tsv", sep='\t')
df_targets.head()

,TF_40,TF_41,TF_42,TF_43,TF_44,TF_45,TF_46,TF_47,TF_48,TF_49,...,C_19,C_18,C_15,C_14,C_17,C_16,C_11,C_10,C_13,C_12
0,823.914118,12702.625538,2124.023125,2314.305782,1474.888697,1131.785521,4597.003319,14589.890994,1556.951404,34180.775942,...,1651.254953,1242.303363,724.105850,3184.883349,8935.394363,12689.558779,4102.312624,505.126184,12946.381724,1313.790253
1,1307.840222,4316.426121,2554.658908,3415.320661,3408.586803,1697.342725,5272.763446,22903.130555,2181.551097,10000.297243,...,3505.604759,2516.000120,1640.114829,3463.713253,19535.468264,18006.721690,6890.427794,1402.597000,38309.856355,3024.107809
2,1188.353499,3436.803941,2088.909658,3708.324021,2219.741833,1571.646567,6225.376501,13858.014077,1971.053716,18800.025304,...,3270.572883,1693.419147,997.792996,3196.992198,16695.027604,14486.992627,13517.968701,10680.866586,25648.825592,2675.530918
3,1806.103795,6531.268855,2406.186212,3601.204703,2828.415329,2746.861783,5810.104650,25701.749693,2191.273065,19213.880658,...,2701.555739,2059.614815,1432.163042,4927.163643,18896.765835,18784.043322,8608.167421,4624.044391,23651.726053,3679.449867
4,1417.411525,3951.243575,2581.309532,3375.884699,2764.716964,1806.919566,5033.976283,26364.859152,2311.790793,16139.097553,...,2457.214141,1901.709222,1672.531034,3877.787322,14699.253953,17119.871513,8995.328144,12641.425965,27999.405431,3128.844808


### Build a dataframe for single transcription factor

In [559]:
def build_df(tf, df_seq, df_targets):
    df = df_seq.copy()
    df["Target"] = df_targets[tf].values
    return df

In [560]:
df = build_df(TRAIN_ON_TF, df_seq, df_targets)
df.head()

,Fold ID,Event ID,seq,Target
0,B,MEreverse14075,TAAAACTATGAGGAAGGATTCAGGGTCGGACAGTGCCTGT,24652.209903
1,B,MEforward19438,CTTATGATCAGAAGCGGCTAGGTGTATTACATGTCCCTGT,23603.659780
2,B,MEforward19439,CCGCCGTAGGCCCCGAAACAGTACCAGACATGTAACCTGT,58487.438924
3,B,MEforward19436,GACCAAACGAGTCCTAGGATTCCAAGCGTTACGACCCTGT,28969.180606
4,B,MEforward19437,CGTTACGACGGAGTTTGGATCCCGAACTTATGATCCCTGT,25171.915989


### Remove probe specific biases for each sequence

In [561]:
biases = df_targets.median(axis=1).values
biases

array([2641.988164 , 4170.2475   , 3699.8877625, ..., 2287.3149815,
       1884.915039 , 1231.9427485], shape=(80856,))

In [562]:
# Note this won't cause data leak because then normalization is sequence specific and does not use other sequences
df['TargetNorm'] = df['Target'].values - biases
df.head()

,Fold ID,Event ID,seq,Target,TargetNorm
0,B,MEreverse14075,TAAAACTATGAGGAAGGATTCAGGGTCGGACAGTGCCTGT,24652.209903,22010.221739
1,B,MEforward19438,CTTATGATCAGAAGCGGCTAGGTGTATTACATGTCCCTGT,23603.659780,19433.412280
2,B,MEforward19439,CCGCCGTAGGCCCCGAAACAGTACCAGACATGTAACCTGT,58487.438924,54787.551161
3,B,MEforward19436,GACCAAACGAGTCCTAGGATTCCAAGCGTTACGACCCTGT,28969.180606,24346.930618
4,B,MEforward19437,CGTTACGACGGAGTTTGGATCCCGAACTTATGATCCCTGT,25171.915989,21294.128667


### Calculate NA content and drop rows

In [563]:
# Only 4% NA content, it is safe to drop the rows
df['Target'].isna().mean()

np.float64(0.012850004947066389)

In [564]:
df.dropna(subset=['Target'], inplace=True)
df['Target'].isna().mean()

np.float64(0.0)

### Train/Test data split

In [565]:
df_train = df[df['Fold ID'] == TRAIN_PROBE]
df_train.head()

,Fold ID,Event ID,seq,Target,TargetNorm
40526,A,HK26479,ACGCGCGTCAGCTTTTTGGAATATTGCGGAGAGTTCCTGT,3690.235644,1035.297721
40527,A,HK26478,GACTATGGGATGGGCCGCCTTTGATTACGCGCGTCCCTGT,10197.241727,7155.318333
40528,A,HK22998,CCTTCGTGAGCCATGTGTTTCAGGCTGTGCGTGTCCCTGT,4336.389547,1962.749611
40529,A,HK22999,GGCGGGTGGTAAAGGCCCCGGAAGCGGACACGCACCCTGT,11108.696660,9423.523044
40530,A,HK26473,GTTGTGGTTTGTCCTTTTGTATTAACAGTGTATGGCCTGT,10835.459353,7094.844587


In [566]:
df_test = df[df['Fold ID'] == TEST_PROBE]
df_test.head()

,Fold ID,Event ID,seq,Target,TargetNorm
0,B,MEreverse14075,TAAAACTATGAGGAAGGATTCAGGGTCGGACAGTGCCTGT,24652.209903,22010.221739
1,B,MEforward19438,CTTATGATCAGAAGCGGCTAGGTGTATTACATGTCCCTGT,23603.659780,19433.412280
2,B,MEforward19439,CCGCCGTAGGCCCCGAAACAGTACCAGACATGTAACCTGT,58487.438924,54787.551161
3,B,MEforward19436,GACCAAACGAGTCCTAGGATTCCAAGCGTTACGACCCTGT,28969.180606,24346.930618
4,B,MEforward19437,CGTTACGACGGAGTTTGGATCCCGAACTTATGATCCCTGT,25171.915989,21294.128667


In [567]:
df_train.shape, df_test.shape

((40063, 5), (39754, 5))

### Add labels for ROC and AUC

In [568]:
# DREAM5 https://pmc.ncbi.nlm.nih.gov/articles/PMC3687085/
def add_label(d):
    # Add positive label to only rows with binding score higher than mean + 4*std
    mean = d['Target'].mean()
    std = d['Target'].std()
    lower_limit = mean + 4*std

    # Limits MAX: 1300 rows MIN: 50 rows
    top = d[d['Target'] > lower_limit].copy()
    if len(top) >= 50:
        top = top.sort_values(by='Target', ascending=False).head(1300)
    else:
        top = d.sort_values(by='Target', ascending=False, inplace=False).head(50)

    d['Label'] = 0
    d.loc[top.index, 'Label'] = 1

In [569]:
add_label(df_train)
df_train.head()

,Fold ID,Event ID,seq,Target,TargetNorm,Label
40526,A,HK26479,ACGCGCGTCAGCTTTTTGGAATATTGCGGAGAGTTCCTGT,3690.235644,1035.297721,0
40527,A,HK26478,GACTATGGGATGGGCCGCCTTTGATTACGCGCGTCCCTGT,10197.241727,7155.318333,0
40528,A,HK22998,CCTTCGTGAGCCATGTGTTTCAGGCTGTGCGTGTCCCTGT,4336.389547,1962.749611,0
40529,A,HK22999,GGCGGGTGGTAAAGGCCCCGGAAGCGGACACGCACCCTGT,11108.696660,9423.523044,0
40530,A,HK26473,GTTGTGGTTTGTCCTTTTGTATTAACAGTGTATGGCCTGT,10835.459353,7094.844587,0


In [570]:
df_train[df_train['Label'] == 1].head()

,Fold ID,Event ID,seq,Target,TargetNorm,Label
40674,A,HK36930,ACCAGATGGGCCCAATTCTCCTGCCTTAAGGTTGGCCTGT,17016.087534,15106.799900,1
41024,A,HK05502,TCCGCGCGGGAACCTAGGCCGCTATTTAACCTAAACCTGT,15340.928216,12716.468335,1
41030,A,HK05508,AGGGCTGTGGATTTTGGGCCTGTCGCGTTCCGATACCTGT,14621.229434,11308.391338,1
41723,A,HK24333,CGGAATTAGGCCTACAGCTGCGCACGTACTCTCTGCCTGT,15065.605272,12237.795894,1
41964,A,HK12672,CCGGATAGGCCCCATTCATCTGCGTGAGCAACGTACCTGT,14946.657754,12438.001401,1


In [571]:
df_train[df_train['Label'] == 1].shape

(125, 6)

In [572]:
add_label(df_test)
df_test.head()

,Fold ID,Event ID,seq,Target,TargetNorm,Label
0,B,MEreverse14075,TAAAACTATGAGGAAGGATTCAGGGTCGGACAGTGCCTGT,24652.209903,22010.221739,0
1,B,MEforward19438,CTTATGATCAGAAGCGGCTAGGTGTATTACATGTCCCTGT,23603.659780,19433.412280,0
2,B,MEforward19439,CCGCCGTAGGCCCCGAAACAGTACCAGACATGTAACCTGT,58487.438924,54787.551161,0
3,B,MEforward19436,GACCAAACGAGTCCTAGGATTCCAAGCGTTACGACCCTGT,28969.180606,24346.930618,0
4,B,MEforward19437,CGTTACGACGGAGTTTGGATCCCGAACTTATGATCCCTGT,25171.915989,21294.128667,0


In [573]:
df_test[df_test['Label'] == 1].head()

,Fold ID,Event ID,seq,Target,TargetNorm,Label
341,B,MEforward20030,ATCTGAGGCCCATATGGTCGGAGACAGTACTATAGCCTGT,68671.537017,64612.614275,1
1487,B,MEforward11688,ATGAAGAGGCCTCGCTTTAGTGTACTGCGGAGGTGCCTGT,68032.254725,64478.573684,1
2016,B,MEforward06388,TTATCGTAGTACTCCCAGGCCCGTATATAGGAGGCCCTGT,69187.251617,65724.472753,1
2987,B,MEforward17176,CTAAGGATCTTAGATTAGTCGTAGGCCTACGTAGGCCTGT,69369.705380,65430.144099,1
3022,B,MEforward01404,GCTAGGCCATGTTTGACACAGGGCCCCTATTCCGCCCTGT,72123.869553,68392.039092,1


In [574]:
df_test[df_test['Label'] == 1].shape

(50, 6)

### Mathmatical target normalization to N(0, 1)

In [575]:
target_norm = df_train['TargetNorm'].values
df_train['TargetNormFinal'] = (target_norm - target_norm.mean()) / target_norm.std()
df_train.head()

,Fold ID,Event ID,seq,Target,TargetNorm,Label,TargetNormFinal
40526,A,HK26479,ACGCGCGTCAGCTTTTTGGAATATTGCGGAGAGTTCCTGT,3690.235644,1035.297721,0,-0.570562
40527,A,HK26478,GACTATGGGATGGGCCGCCTTTGATTACGCGCGTCCCTGT,10197.241727,7155.318333,0,1.934831
40528,A,HK22998,CCTTCGTGAGCCATGTGTTTCAGGCTGTGCGTGTCCCTGT,4336.389547,1962.749611,0,-0.190885
40529,A,HK22999,GGCGGGTGGTAAAGGCCCCGGAAGCGGACACGCACCCTGT,11108.696660,9423.523044,0,2.863381
40530,A,HK26473,GTTGTGGTTTGTCCTTTTGTATTAACAGTGTATGGCCTGT,10835.459353,7094.844587,0,1.910074


In [576]:
df_test['TargetNormFinal'] = (df_test['TargetNorm'].values - target_norm.mean()) / target_norm.std()
df_test.head()

,Fold ID,Event ID,seq,Target,TargetNorm,Label,TargetNormFinal
0,B,MEreverse14075,TAAAACTATGAGGAAGGATTCAGGGTCGGACAGTGCCTGT,24652.209903,22010.221739,0,8.016079
1,B,MEforward19438,CTTATGATCAGAAGCGGCTAGGTGTATTACATGTCCCTGT,23603.659780,19433.412280,0,6.961194
2,B,MEforward19439,CCGCCGTAGGCCCCGAAACAGTACCAGACATGTAACCTGT,58487.438924,54787.551161,0,21.434348
3,B,MEforward19436,GACCAAACGAGTCCTAGGATTCCAAGCGTTACGACCCTGT,28969.180606,24346.930618,0,8.972673
4,B,MEforward19437,CGTTACGACGGAGTTTGGATCCCGAACTTATGATCCCTGT,25171.915989,21294.128667,0,7.722928


### DNA/RNA sequence to Matrix logic

In [577]:
def fill_cell(motif_len, row, col, bases, seq):
    num_rows = len(seq) + 2 * motif_len - 2
    
    # First M-1 rows are filled with 0.25
    if row < motif_len-1:
        return 0.25

    # Last M-1 rows are filled with 0.25
    if num_rows-1-row < motif_len-1:
        return 0.25

    idx = row - motif_len + 1
    if seq[idx] == bases[col]:
        return 1.0

    return 0.0

def seq2matrix(seq, motif_len, typ='DNA'):
    bases = DNA_BASES if typ == 'DNA' else RNA_BASES
    num_rows = len(seq) + 2 * motif_len - 2
    result = np.empty([num_rows, 4])
    for row in range(num_rows):
        for col in range(4):
            result[row, col] = fill_cell(motif_len, row, col, bases, seq)
    return np.transpose(result)

In [578]:
# Test the function
S = seq2matrix("ATGG", 3, 'DNA')
S

array([[0.25, 0.25, 1.  , 0.  , 0.  , 0.  , 0.25, 0.25],
       [0.25, 0.25, 0.  , 0.  , 0.  , 0.  , 0.25, 0.25],
       [0.25, 0.25, 0.  , 0.  , 1.  , 1.  , 0.25, 0.25],
       [0.25, 0.25, 0.  , 1.  , 0.  , 0.  , 0.25, 0.25]])

In [579]:
S.shape

(4, 8)

### Train Data Augmentation (Reverse Complement)

In [580]:
def reverse_complement_batch(x):
    # x: (B, 4, L)
    # reverse the sequence
    x = torch.flip(x, dims=[2])
    
    # swap A-T, C-G
    # A C G T → T G C A
    x = x[:, [3, 2, 1, 0], :]

    return x

def reverse_complement_M(x):
    # x: (4, L)
    # reverse the sequence
    x = torch.flip(x, dims=[1])
    
    # swap A-T, C-G
    # A C G T → T G C A
    x = x[[3, 2, 1, 0], :]

    return x

### Sequence Dataset and Loader

In [581]:
class SeqDataset(Dataset):
    def __init__(self, df, augment=False):
        self.sequences = df['seq'].values
        self.targets = df['TargetNormFinal'].values
        self.labels = df['Label'].values
        self.aug = augment

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        
        M = seq2matrix(seq, MOTIF_LEN, 'DNA')
        
        x = torch.tensor(M, dtype=torch.float32)
        y = torch.tensor(self.targets[idx], dtype=torch.float32)
        label = self.labels[idx].copy()

        # Augment the data
        if self.aug and random.random() > 0.5:
            x = reverse_complement_M(x)
        
        return x, y, label

In [582]:
train_dataset = SeqDataset(df_train, augment=True)
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

In [583]:
xs, ys, labels = [], [], []

for x, y, label in train_dataset:
    xs.append(x)
    ys.append(y)
    labels.append(label)

x_train = torch.stack(xs)
y_train = torch.tensor(ys, dtype=torch.float32)
label_train = torch.tensor(labels)

In [584]:
x, target, label = next(iter(train_loader))
x.shape, target.shape, label.shape

(torch.Size([64, 4, 102]), torch.Size([64]), torch.Size([64]))

In [585]:
# Do NOT augment test dataset
test_dataset = SeqDataset(df_test, augment=False)

xs, ys, labels = [], [], []

for x, y, label in test_dataset:
    xs.append(x)
    ys.append(y)
    labels.append(label)

x_test = torch.stack(xs)
y_test = torch.tensor(ys, dtype=torch.float32)
label_test = torch.tensor(labels)

In [586]:
x_test.shape, y_test.shape, label_test.shape

(torch.Size([39754, 4, 102]), torch.Size([39754]), torch.Size([39754]))

# Model

### DeepBind Model

In [587]:
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

class DeepBindShallow(nn.Module):
    def __init__(self, 
                 num_motif_detectors, 
                 motif_len,
                 dropout,
                 conv_init_w,
                 fc_init_w):
        super().__init__()

        self.conv = nn.Conv1d(in_channels=4, out_channels=num_motif_detectors, kernel_size=motif_len)
        self.dropout = nn.Dropout(p=dropout)
        self.fc1 = nn.Linear(num_motif_detectors, 1)
        
        self.init_weights(conv_init_w, fc_init_w)

    def init_weights(self, conv_init_w, fc_init_w):
        init.normal_(self.conv.weight, mean=0.0, std=conv_init_w)
        init.constant_(self.conv.bias, 0.0)

        init.normal_(self.fc1.weight, mean=0.0, std=fc_init_w)
        init.constant_(self.fc1.bias, 0.0)

    def forward_pass(self, x):
        
        x = self.conv(x)
        x = F.relu(x)
        x, _ = torch.max(x, dim=2)
        x = self.dropout(x)
        x = self.fc1(x)

        return x

    def forward(self, x):
        r = self.forward_pass(x)
        
        x_comp = reverse_complement_batch(x)
        r_comp = self.forward_pass(x_comp)
        
        r_final = torch.max(r, r_comp)
        
        return r_final

In [588]:
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

class DeepBind(nn.Module):
    def __init__(self, 
                 num_motif_detectors, 
                 motif_len,
                 dropout,
                 conv_init_w,
                 fc_init_w):
        super().__init__()

        self.conv = nn.Conv1d(in_channels=4, out_channels=num_motif_detectors, kernel_size=motif_len)
        self.dropout1 = nn.Dropout(p=dropout)
        self.fc1 = nn.Linear(num_motif_detectors, 32)
        self.dropout2 = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(32, 1)

        self.init_weights(conv_init_w, fc_init_w)

    def init_weights(self, conv_init_w, fc_init_w):
        init.normal_(self.conv.weight, mean=0.0, std=conv_init_w)
        init.constant_(self.conv.bias, 0.0)

        init.normal_(self.fc1.weight, mean=0.0, std=fc_init_w)
        init.constant_(self.fc1.bias, 0.0)
        
        init.normal_(self.fc2.weight, mean=0.0, std=fc_init_w)
        init.constant_(self.fc2.bias, 0.0)

    def forward_pass(self, x):
        
        x = self.conv(x)
        x = F.relu(x)
        x, _ = torch.max(x, dim=2)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)

        return x

    def forward(self, x):
        r = self.forward_pass(x)
        
        x_comp = reverse_complement_batch(x)
        r_comp = self.forward_pass(x_comp)
        
        r_final = torch.max(r, r_comp)
        
        return r_final

### Model Wrapper for training

In [589]:
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

class ModelWrapper:
    def __init__(self, m, lr=1e-3, w_decay=1e-3):
        self.model = m.to(DEVICE)
        #self.opt = torch.optim.Adam(self.model.parameters(), lr=0.0005)
        self.opt = optim.AdamW(self.model.parameters(), lr=lr, weight_decay=w_decay)
        
        self.criterion = torch.nn.MSELoss()

    def train_step(self, x, target):
        self.model.train()
        
        x = x.to(DEVICE)
        target = target.to(DEVICE)

        self.opt.zero_grad()

        pred = self.model(x)
        loss = self.criterion(pred.squeeze(), target)

        loss.backward()
        self.opt.step()
        
        return loss.item()

    def predict(self, x):
        self.model.eval()
        x = x.to(DEVICE)
        pred = self.model(x).squeeze()
        pred = pred.detach().cpu().numpy()
        return pred

    def evaluate(self, x, y_true, label_true):
        y_pred = self.predict(x)

        fpr, tpr, thresholds = roc_curve(label_true, y_pred)
        roc_auc = auc(fpr, tpr)
        
        pearson_corr, _ = pearsonr(y_true, y_pred)
        spearman_corr, _ = spearmanr(y_true, y_pred)

        # plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
        # plt.plot([0, 1], [0, 1], linestyle='--')
        # plt.xlabel("False Positive Rate")
        # plt.ylabel("True Positive Rate")
        # plt.legend()
        # plt.show()

        return {
            'pearson': pearson_corr,
            'spearman': spearman_corr,
            'auc': roc_auc,
        }

    def train_one_epoch(self, loader):
        epoch_loss = 0
        for i, (x, y, label) in enumerate(tqdm(loader)):
            epoch_loss += self.train_step(x, y)
        epoch_loss = epoch_loss / len(loader)
        return epoch_loss

### Sanity Check: Overfit on single mini-batch

In [590]:
x, y, label = next(iter(train_loader))
x.shape, y.shape, label.shape

(torch.Size([64, 4, 102]), torch.Size([64]), torch.Size([64]))

In [591]:
m_sanity = DeepBindShallow(num_motif_detectors=16, motif_len=24, dropout=0.95, conv_init_w=0.001, fc_init_w=0.001)

In [592]:
mw_sanity = ModelWrapper(m_sanity)

In [544]:
for i in range(10000):
    loss = mw_sanity.train_step(x, y)
    if i % 1000 == 0:
        print(loss)

1.5963393449783325
1.333681344985962
0.8059917688369751
1.4180238246917725
1.2546128034591675
1.347851037979126
0.6801857948303223
0.7549934983253479
0.7918535470962524
1.4197611808776855


In [545]:
pred_sanity = mw_sanity.predict(x)
pred_sanity[:10]

array([-1.3913959e-03,  3.9298367e-02,  5.8030114e-02,  1.6137331e+00,
        1.7538747e-01, -2.2062328e-02, -1.1360131e-02, -1.5314184e-02,
       -3.7912540e-03, -2.8610535e-02], dtype=float32)

In [546]:
target_sanity = y.cpu().numpy()
target_sanity[:10]

array([-0.27307722,  0.08115514,  0.2736761 ,  6.3071117 ,  0.6529401 ,
       -0.5904817 , -0.66178095, -0.55244404, -0.6513517 , -0.62807626],
      dtype=float32)

In [547]:
for name, p in mw_sanity.model.named_parameters():
    print(name, p.data.abs().mean().item())

conv.weight 0.06308583170175552
conv.bias 0.017959177494049072
fc1.weight 0.10577191412448883
fc1.bias 0.03392793983221054


### Sanity Check: Reverse compliment

In [548]:
np.allclose(mw_sanity.predict(x), mw_sanity.predict(reverse_complement_batch(reverse_complement_batch(x))))

True

# Training

In [593]:
import os

def save_snapshot(mw, epoch, params):
    snap_path = "/tmp/snaps"
    os.makedirs(snap_path, exist_ok=True)
    name = f"{snap_path}/model_epoch_{epoch}.pt"
    torch.save({
        "model": mw.model.state_dict(),
        "params": params,
    }, name)
    return name

def load_snapshot(path):
    ckpt = torch.load(path, map_location="cpu")
    params = ckpt["params"]
    print(params)

    m = DeepBind(
        num_motif_detectors=params['num_motif_detectors'],
        motif_len=params['motif_len'],
        dropout=params['dropout'],
        conv_init_w=params['conv_init_weight_std'],
        fc_init_w=params['fc_init_weight_std'],
    )
    mw = ModelWrapper(m, lr=params['lr'], w_decay=params['weight_decay'])
    
    mw.model.load_state_dict(ckpt["model"])

    return mw

def load_snapshot_shallow(path):
    ckpt = torch.load(path, map_location="cpu")
    params = ckpt["params"]
    print(params)

    m = DeepBindShallow(
        num_motif_detectors=params['num_motif_detectors'],
        motif_len=params['motif_len'],
        dropout=params['dropout'],
        conv_init_w=params['conv_init_weight_std'],
        fc_init_w=params['fc_init_weight_std'],
    )
    mw = ModelWrapper(m, lr=params['lr'], w_decay=params['weight_decay'])
    
    mw.model.load_state_dict(ckpt["model"])

    return mw

In [594]:
import wandb

def train_wandb(run_name, mw, loader, epochs, params):
    run = wandb.init(
        entity="vvaza22-free-university-of-tbilisi",
        project="CompBioFinal",
        name=run_name,
        config={
            "model": "DeepBind",
            "dataset": "DREAM5",
            "epochs": epochs,
            **params
        },
    )

    for epoch in range(1, epochs+1):
        print(f"===== EPOCH {epoch} =====")
        epoch_loss = mw.train_one_epoch(loader)
        print(f"Loss: {epoch_loss}")

        metrics_train = mw.evaluate(x_train, y_train, label_train)
        metrics_test = mw.evaluate(x_test, y_test, label_test)
        
        wandb.log({
            'epoch_loss': epoch_loss,
            'train/pearson': metrics_train['pearson'],
            'train/spearman': metrics_train['spearman'],
            'train/auc': metrics_train['auc'],
            'test/pearson': metrics_test['pearson'],
            'test/spearman': metrics_test['spearman'],
            'test/auc': metrics_test['auc'],
        }, step=epoch)

    run.finish()

In [597]:
import wandb

MIN_DELTA = 1e-3

def early_stopping_train_wandb(run_name, mw, loader, epochs, params):
    run = wandb.init(
        entity="vvaza22-free-university-of-tbilisi",
        project="CompBioBaseline__TF_31",
        name=run_name,
        config={
            "model": "DeepBind",
            "dataset": "DREAM5",
            "epochs": epochs,
            "tf": TRAIN_ON_TF,
            **params
        },
    )

    best_test_pearson = 0

    for epoch in range(1, epochs+1):
        print(f"===== EPOCH {epoch} =====")
        epoch_loss = mw.train_one_epoch(loader)
        print(f"Loss: {epoch_loss}")

        metrics_train = mw.evaluate(x_train, y_train, label_train)
        metrics_test = mw.evaluate(x_test, y_test, label_test)

        results = {
            'epoch_loss': epoch_loss,
            'train/pearson': metrics_train['pearson'],
            'train/spearman': metrics_train['spearman'],
            'train/auc': metrics_train['auc'],
            'test/pearson': metrics_test['pearson'],
            'test/spearman': metrics_test['spearman'],
            'test/auc': metrics_test['auc'],
        }
        print(results)

        # Snapshot the model
        snap = save_snapshot(mw, epoch, params)
        artifact = wandb.Artifact(
            name=f"model_epoch_{epoch}",
            type="model"
        )
        artifact.add_file(snap)
        wandb.log_artifact(artifact)
        
        wandb.log(results, step=epoch)

        if np.isnan(metrics_test['pearson']):
            # Vanishing Gradient? something went very wrong
            break

        if metrics_test['pearson'] > best_test_pearson:
            best_test_pearson = metrics_test['pearson']
        elif metrics_test['pearson'] + MIN_DELTA < best_test_pearson:
            # Overfitting
            print(f"Early stopping on epoch {epoch}")
            break

    run.finish()

In [598]:
from sklearn.model_selection import ParameterGrid
from scipy.stats import loguniform

param_grid = {
    'lr': [0.0005],
    'dropout': [0.5],
    'num_motif_detectors': [16],
    'motif_len': [12, 16, 24],
    'weight_decay': [1e-3, 5e-3],
    'conv_init_weight_std': [1e-3, 1e-4],
    'fc_init_weight_std': [1e-2, 1e-3],
}
print(param_grid)

grid = ParameterGrid(param_grid)

print(f"Total: {len(grid)}")

{'lr': [0.0005], 'dropout': [0.5], 'num_motif_detectors': [16], 'motif_len': [12, 16, 24], 'weight_decay': [0.001, 0.005], 'conv_init_weight_std': [0.001, 0.0001], 'fc_init_weight_std': [0.01, 0.001]}
Total: 24


In [599]:
run_id = 1
for params in grid:
    print(f"Calibration with: {params}")

    run_name = f"{TRAIN_ON_TF}_Best_Calib_{run_id}"
    
    m = DeepBind(
        num_motif_detectors=params['num_motif_detectors'],
        motif_len=params['motif_len'],
        dropout=params['dropout'],
        conv_init_w=params['conv_init_weight_std'],
        fc_init_w=params['fc_init_weight_std'],
    )
    mw = ModelWrapper(m, lr=params['lr'], w_decay=params['weight_decay'])

    early_stopping_train_wandb(run_name, mw, train_loader, 50, params=params)

    run_id = run_id + 1

Calibration with: {'conv_init_weight_std': 0.001, 'dropout': 0.5, 'fc_init_weight_std': 0.01, 'lr': 0.0005, 'motif_len': 12, 'num_motif_detectors': 16, 'weight_decay': 0.001}


===== EPOCH 1 =====


  0%|          | 0/626 [00:00<?, ?it/s]

Loss: 0.7069444726831235
{'epoch_loss': 0.7069444726831235, 'train/pearson': np.float32(0.86206955), 'train/spearman': np.float64(0.6732880220166023), 'train/auc': 0.9563555511042116, 'test/pearson': np.float32(0.8023156), 'test/spearman': np.float64(0.5213805637378037), 'test/auc': 0.9616617973000202}
===== EPOCH 2 =====


  0%|          | 0/626 [00:00<?, ?it/s]

...

  0%|          | 0/626 [00:00<?, ?it/s]

Loss: 0.29264657313640885
{'epoch_loss': 0.29264657313640885, 'train/pearson': np.float32(0.90690154), 'train/spearman': np.float64(0.6852060478851444), 'train/auc': 0.9728915819520256, 'test/pearson': np.float32(0.8452103), 'test/spearman': np.float64(0.5689729612687193), 'test/auc': 0.9706553495869434}
===== EPOCH 11 =====


  0%|          | 0/626 [00:00<?, ?it/s]

Loss: 0.2992774634220349
{'epoch_loss': 0.2992774634220349, 'train/pearson': np.float32(0.90569854), 'train/spearman': np.float64(0.6811050498166905), 'train/auc': 0.9731638038960388, 'test/pearson': np.float32(0.8436426), 'test/spearman': np.float64(0.5677288432475684), 'test/auc': 0.9731649204110417}
Early stopping on epoch 11


epoch_loss,█▁▁▁▁▁▁▁▁▁▁
test/auc,▁▄▇▅▄▇▅▅▆▆█
test/pearson,▁▄▅▅▆▇▇▇▇█▇
test/spearman,▁▃▅▆▆▇▇▇▇██
train/auc,▁▄▄▅▆▇▆▆▇██
train/pearson,▁▃▅▅▆▆▇▇▇█▇
train/spearman,▁▃▅▆▆▇▇▇▇██
epoch_loss,0.29928
test/auc,0.97316
test/pearson,0.84364
test/spearman,0.56773


In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('vvaza22-free-university-of-tbilisi/CompBioFinal/model_epoch_25:v72', type='model')
artifact_dir = artifact.download()

In [ ]:
path = f"{artifact_dir}/model_epoch_25.pt"
path

In [ ]:
mw = load_snapshot(path)

In [ ]:
mw.evaluate(x_test, y_test, label_test)

### Build PWM matrix from Conv Layer

In [ ]:
conv_weights = mw.model.conv.weight.detach().cpu().numpy()
conv_weights.shape

In [ ]:
def filter_to_pwm(conv_filter):
    pwm = np.exp(conv_filter)
    pwm = pwm / pwm.sum(axis=0, keepdims=True)
    return pwm

def filter_to_pwm_df(conv_filter):
    pwm = filter_to_pwm(conv_filter)
    return pd.DataFrame(pwm.T, columns=['A','C','G','T'], dtype='float64')

In [ ]:
pwm_df = filter_to_pwm_df(conv_weights[15])
pwm_df

In [ ]:
info_df = logomaker.transform_matrix(pwm_df, from_type='probability', to_type='information')
info_df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3))
    
logo = logomaker.Logo(info_df,
                      ax=ax,
                      color_scheme='classic',
                      vpad=.1,
                      width=.8)

logo.style_spines(visible=False)
logo.style_spines(spines=['left', 'bottom'], visible=True)
ax.set_ylabel("Information (Bits)")
ax.set_xlabel("Position in Motif")
#ax.set_ylim([0, 2])

plt.show()